In [1]:
# для работы с русским текстом необходимо установить pymorphy2
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=9b81e240c6e43238852151e9bd07447e89646c0fb653404642f2f084322a20dc
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [2]:
# для сравнения расстояния между последовательностями установим библиотеку textdistance
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [262]:
# импортируем неободимые библиотеки
import re
import os
from pymorphy2 import MorphAnalyzer
import nltk
import textdistance
import glob
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [139]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [11]:
# загрузим необхоимые компоненты nltk
nltk.download('punkt')      # для токенизации
nltk.download('stopwords')  # для фильтрации на стоп-лова

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [269]:
# определим функцию проводящую токенизации и лемматизации текста, которая возвращет список из слов
def get_words_tok(text):
  tokenized = list()
  morph = MorphAnalyzer()
  word_text = nltk.word_tokenize(text)
  for i in word_text:
    txt = re.findall(r"[а-яё]+", i.lower())
    for j in txt:
      if j not in stopwords.words("russian") and len(j) > 2:
        tokenized.append(morph.parse(j)[0].normal_form)
  return tokenized

In [264]:
# определим функцию проводящую токенизации и лемматизации текста, которая возвращает список из предложений
def get_sent_tok(text):
  tokenized = list()
  morph = MorphAnalyzer()
  sent_text = nltk.sent_tokenize(text)
  for i in sent_text:
    tok_sen = []
    txt = re.findall(r"[а-яё]+", i.lower())
    for j in txt:
      if j not in stopwords.words("russian") and len(j) > 2:
        tok_sen.append(morph.parse(j)[0].normal_form)
    tokenized.append(" ".join(tok_sen))
  return tokenized

In [253]:
# определим функцию для чтения файла
def get_file(link):
  with open(link, "r", encoding="utf-8") as file:
    return file.read()

In [266]:
# оперделим функию для сохранения файла
"""
функция имеет один именованный параметр flag, 
в случае если передать в функцию аргумент flag=True на первой строке файла добавится
информация о сходстве данного файла с файлом из нашей базы_даных_курсовых_работ
функция универсальна и может сохранять как список так и строку
"""
def save_file(link, text, flag=False):
  with open(link, "w", encoding="utf-8") as file:
    if flag:
      file.write(f"содержит {result[1]}% сходства с работой '{file_name_2}'\n")
    if type(text) is list:
      for i in text:
        file.write(i)
        file.write(" ")
    else:
      file.write(text)

In [261]:
# определим функцию проверки на плагиат
"""
В функции присутсвтует 4 вида метрик для определния сходства между текстами.
Мы будем использовать косинусное сходство
"""
def get_plagiarism(text_1, text_2):
  cosine = textdistance.cosine(text_1, text_2)
  # bag = textdistance.bag(text_1, text_2)
  # levenshtein = textdistance.levenshtein(text_1, text_2)
  # jaccard = textdistance.jaccard(text_1, text_2)
  # данный критерий можно ужесточить и возвращать True при меньшем сходстве
  return cosine > 0.30, round(cosine * 100)

In [181]:
# для начала обработаем каждый текст и приведём его в читабельный для машины вид
# данная операция выполняется единожды для побработки  первого файла, далее программа будет обращаться к папку обработанные_работы
way_to_files = glob.glob("/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/база_данных_курсовых_работ/*.txt")

In [182]:
# посмотрим верно ли отработал glob
way_to_files

['/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/база_данных_курсовых_работ/1.структурное_программирование.txt']

In [230]:
# проведем предобработку текстов базы курсовых работ
for way in way_to_files:
  text = get_file(way)
  text = get_words_tok(text)
  
  # чтобы не менять исходные файлы, сохраним предобработанные файлы в другрую директорию
  new_way = f"/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/обработанные_работы{way[way.rfind('/'):]}"
  save_file(new_way, text)

In [270]:
# теперь прочитаем наш проверяемый текст
# напишем код, который сможет проверять, если это необходимо несколько текстов.
links_1 = glob.glob("/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/проверяемые_работы/*.txt")
for link_1 in links_1:
  # прочитаем проверяемый файл
  text_check = get_file(link_1)

  # токенизируем его
  text_1 = get_words_tok(text_check)

  # имя первого файла для удобства
  file_name_1 = f"{link_1[link_1.rfind('/')+1:]}"

  # если в папку обработанные_работы добавятся новые файлы, то при заходе на внутренний цикл нужно заново получить ссылку на файлы этой папки
  links_2 = glob.glob("/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/обработанные_работы/*.txt")
  # список в котором хранятся порядковые номера файлов
  count_file = list(map(lambda x: str(x)[x.rfind("/") + 1:x.find(".")], links_2))

  for link_2 in links_2:
    # имя второго файла для удобства
    file_name_2 = f"{link_2[link_2.rfind('/')+1:]}"
    # прочитаем файл из базы данных в список слов
    text_2 = get_file(link_2).split()
    # сравним два файла
    result = get_plagiarism(text_1, text_2)
    # если файлы схожи ,более чем на 30%  файл считается плагиатом
    if result[0]:
      print(f"""Обнаружен плагиат!
Проверяемая работа '{file_name_1}' содержит {result[1]}% сходства с работой '{file_name_2}'
Исходный файл перемещён в папку 'плагиат'""")
      save_file(f"/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/плагиат/{file_name_1}", text_check, flag=True)
      os.remove(link_1)
      break
  # необязательный оператор условный else цикла for. В случае не штатного завершения цикла, оператор не сработает
  else:
    """
    получаем новое имя файла для сохранения на диск. для этого используем:
    * max() для поиска наибольшего порядкового номера
    * map() для изменения значения строки на целое списка count_file
    * lambda меняет это значение
    """
    new_file_name = f"{max(map(lambda x: int(x), count_file)) + 1}.{link_1[link_1.rfind('/')+1:]}"
    link_3 = "/content/drive/MyDrive/programming/МЭО/Lessons/итоговая_аттестация/база_данных_курсовых_работ/"
    print(f"""Плагиат не обнаружен!
Исходный файл '{file_name_1}' удалён из папки 'проверяемые_работы' и сохранён в папке 'база_данных_курсовых_работ'
Предобработанный файл '{new_file_name}' сохранён в папке 'обработанные_работы'""")
    save_file(f"{link_3}{new_file_name}", text_check)
    save_file(f"{link_2[:link_2.rfind('/')]}/{new_file_name}", text_1)
    os.remove(link_1)